# 1. Import

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

import logging
import random
import warnings
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import StratifiedShuffleSplit
from torch import nn
from torchvision import models, transforms
from torchvision.models import resnet, densenet, squeezenet
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD
import torch.nn.functional as F
from skimage.io import imread
import matplotlib.pyplot as plt
%matplotlib inline
import math
# for creating validation set
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import pdb
torch.set_printoptions(linewidth=120)

In [ ]:
warnings.simplefilter('ignore')

# 2.Data Load && Preprocess

In [ ]:
train_images = pd.read_pickle('/kaggle/input/modified-mnist/train_max_x') 
test_images = pd.read_pickle('/kaggle/input/modified-mnist/test_max_x')
train_label = pd.read_csv('/kaggle/input/modified-mnist/train_max_y.csv')

In [ ]:
print(train_images.shape)
print(train_label.shape)
print(test_images.shape)

In [ ]:
plot_size = plt.rcParams["figure.figsize"] 
plot_size[0] = 8
plot_size[1] = 6
plt.rcParams["figure.figsize"] = plot_size 
train_label.Label.value_counts().plot(kind='pie',autopct='%1.0f')

In [ ]:
plt.rcParams['figure.figsize'] = (8, 5)
plt.bar(train_label['Label'].value_counts().index, train_label['Label'].value_counts())
plt.xticks(np.arange(10))
plt.xlabel('Class', fontsize=16)
plt.ylabel('Count', fontsize=16)
plt.grid('on', axis='y')

## 2.2 Gaussian Filter

In [ ]:
def filter_image(img):
    image = np.array(img, dtype=np.uint8)
    image = (255-image)
    kernel = np.ones((1,1),np.uint8)

    median = cv2.medianBlur(image, 1)

    thresh = cv2.threshold(median.copy(), 60, 255, cv2.THRESH_BINARY)[1]
    
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)

    return opening

In [ ]:
from torchvision.utils import make_grid

random_sel = np.random.randint(len(train_images), size=8)

grid = make_grid(torch.Tensor((train_images[random_sel]/255).reshape((-1,128,128))).unsqueeze(1), nrow=8)
plt.rcParams['figure.figsize'] = (16,2)
plt.imshow(grid.numpy().transpose((1,2,0)))
plt.suptitle('Random samples before filtering')
plt.axis('off')
print(*list(train_label.iloc[random_sel, 0].values), sep=', ')

In [ ]:
import cv2
for i in range(0, len(train_images)):
    train_images[i] = filter_image(train_images[i])
for i in range(0, len(test_images)):
    test_images[i] = filter_image(test_images[i])

In [ ]:
grid = make_grid(torch.Tensor((train_images[random_sel]/255).reshape((-1,128,128))).unsqueeze(1), nrow=8)
plt.rcParams['figure.figsize'] = (16,2)
plt.imshow(grid.numpy().transpose((1,2,0)))
plt.suptitle('Random samples after filtering')
plt.axis('off')

## 2.1 Normalization

In [ ]:
train_images = train_images/255

In [ ]:
grid = make_grid(torch.Tensor((train_images[random_sel]).reshape((-1,128,128))).unsqueeze(1), nrow=8)
plt.rcParams['figure.figsize'] = (16,2)
plt.imshow(grid.numpy().transpose((1,2,0)))
plt.suptitle('Random samples after Normalization')
plt.axis('off')

In [ ]:
import gc 
gc.collect()

## 2.2 Split && loader packing

In [ ]:
# preprocessing and formatting
x_train = np.floor(train_images).reshape(-1, 128, 128)
y_train = train_label.Label.values.reshape(-1)
x_test  = np.floor(test_images).reshape(-1, 128, 128)

# training-validation split
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2)   

x_train = torch.from_numpy(x_train).float().unsqueeze(1)
y_train = torch.from_numpy(y_train).long()

x_valid = torch.from_numpy(x_valid).float().unsqueeze(1)
y_valid = torch.from_numpy(y_valid).long()

x_test  = torch.from_numpy(x_test).float().unsqueeze(1)

In [ ]:
x_train.shape, y_train.shape, x_valid.shape, y_valid.shape

In [ ]:
import torch.utils.data as Data
# create torch dataset and dataloader
train_dataset = Data.TensorDataset(x_train, y_train)
train_loader  = Data.DataLoader(train_dataset, batch_size=64, shuffle=True)

valid_dataset = Data.TensorDataset(x_valid, y_valid)
valid_loader  = Data.DataLoader(valid_dataset, batch_size=64, shuffle=True)

test_loader   = Data.DataLoader(x_test, batch_size=64, shuffle=False)

In [ ]:
# (x_train.shape, y_train.shape), (val_x.shape, val_y.shape)

In [ ]:
# train_x = x_train.reshape(40000,1,128,128)
# train_x = torch.from_numpy(train_x)
# train_y = y_train.astype(int)
# train_y = torch.from_numpy(train_y)
# train_x.shape, train_y.shape

In [ ]:
# val_x = val_x.reshape(10000,1,128,128)
# val_x = torch.from_numpy(val_x)
# val_y = val_y.astype(int)
# val_y = torch.from_numpy(val_y)
# val_x.shape, val_y.shape

In [ ]:
# import torch.utils.data as utils
# train_data = utils.TensorDataset(train_x,train_y) # create your datset
# test_data = utils.TensorDataset(val_x,val_y) # create your datset

In [ ]:
# train_loader = utils.DataLoader(train_data,batch_size = 64) # create your dataloader
# test_loader = utils.DataLoader(test_data,batch_size = 64) # create your dataloader

In [ ]:
import gc 
gc.collect()

# 3. Models

## 3.1 Model 1

In [ ]:
class Net1(nn.Module): 
    def __init__(self):
        super(Net1, self).__init__()
          
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True), 
            nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
        )
          
        self.classifier = nn.Sequential(
            nn.Dropout(p = 0.5),
            nn.Linear(4096, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(p = 0.5),
            nn.Linear(512, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(p = 0.5),
            nn.Linear(512, 10),
        )
          
        for m in self.features.children():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
        
        for m in self.classifier.children():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform(m.weight)
            elif isinstance(m, nn.BatchNorm1d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
                

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        
        return x     

### 3.3 random test to validate if the model was well defined

In [ ]:
test_conv_net = Net1()
test_x = torch.rand(2, 1, 128, 128)
test_y = test_conv_net(test_x)

In [ ]:
print(test_y)

# 4. Helper Functions

In [ ]:
def train(epoch):
    model.train()
    exp_lr_scheduler.step()
    train_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        
        if torch.cuda.is_available():
            data = data.cuda()
            target = target.cuda()
        
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        train_loss += loss.item()
        
        loss.backward()
        optimizer.step()
        
        if (batch_idx + 1)% 100 == 0:
            print('Train Epoch: {}| lr: {}, |Batch: [{}/{} ({:.0f}%)] \t | Loss: {:.6f}'.format(
                epoch,
                exp_lr_scheduler.get_lr()[0],
                (batch_idx + 1) * len(data),
                len(train_loader.dataset),
                100. * (batch_idx + 1) / len(train_loader),
                loss.data))
    tr_loss.append(train_loss / (batch_idx+1))

In [ ]:
def evaluate(data_loader):
    model.eval()
    loss = 0
    correct = 0
    for data, target in data_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        if torch.cuda.is_available():
            data = data.cuda()
            target = target.cuda()
        
        output = model(data)
        
        loss += F.cross_entropy(output, target, size_average=False).data

        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
    loss /= len(data_loader.dataset)
        
    print('\nAverage loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)\n'.format(
        loss, 
        correct, 
        len(data_loader.dataset),
        100. * correct / len(data_loader.dataset)))
    val_loss.append(loss.item())
    val_acc.append((100. * correct / len(data_loader.dataset)).item())

# 5. Training

## 5.1 StandCNN

In [ ]:
model = Net1()

optimizer = torch.optim.Adam(model.parameters(),lr=0.01)

criterion = nn.CrossEntropyLoss()

exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7,gamma=0.1)

if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()
print(model)

In [ ]:
n_epochs = 50
epoch_nums = []
tr_loss = []
val_loss = []
val_acc = []
for epoch in range(n_epochs):
    train(epoch)
    evaluate(valid_loader)
    epoch_nums.append(epoch)

## save the model

In [ ]:
checkpoint = {'model': Net1(),
              'state_dict': model.state_dict(),
              'optimizer' : optimizer.state_dict()}

torch.save(checkpoint, 'CNN13.pth')

In [ ]:
from IPython.display import FileLink
FileLink(r'CNN13.pth')

In [ ]:
len(epoch_nums),len(tr_loss),len(val_loss), len(val_acc)

In [ ]:
final_res = pd.DataFrame({
                'Epochs': epoch_nums,
                'train loss':tr_loss,
                'validation loss': val_loss,
                'validation Accuracy': val_acc
})

In [ ]:
final_res.to_csv('CNN13.csv',index=False)
FileLink(r'CNN13.csv')